In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import pandas as pd
import time
from datetime import datetime

# --- PARÁMETROS ESPECÍFICOS PARA TU TAREA ---
GENRES = ['reggaeton']
YEARS = [2000, 2024] 

In [3]:
pip install python-dotenv

In [4]:
import os
from dotenv import load_dotenv

load_dotenv() # Carga las variables del archivo .env

SPOTIPY_CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')
LASTFM_API_KEY = os.getenv('LASTFM_API_KEY')


In [5]:
# Años para iterar
YEARS = [2000, 2024] 

# Términos optimizados para una búsqueda de ALTO VOLUMEN
# Incluye etiquetas históricas ('dembow') y modernas ('trap latino', 'urbano latino')
query_terms = [
    'reggaeton',
    'regueton',
    'urbano latino',
    'perreo',
    'reggaeton pop',
    'latin urban',
    'pop latino',
    'trap latino',
    'urbano latino',
    'dembow',
    '"hip hop latino"' # Entre comillas para buscar la frase exacta (útil para el 2000)
]

# 4. Inicializa el cliente de Spotify
try:
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET
    ))
    print("✅ Conexión a Spotify exitosa.")
except Exception as e:
    print(f"❌ Error de conexión a Spotify: {e}")
    sp = None

❌ Error de conexión a Spotify: No client_id. Pass it or set a SPOTIPY_CLIENT_ID environment variable.


In [5]:
# NUEVA CELDA: Comprobación de autenticación
print(type(sp)) 

<class 'spotipy.client.Spotify'>


In [1]:
def extract_spotify_reggaeton_ampliada(
    sp,
    years,
    query_terms,
    max_total_songs=500,
    max_songs_per_year=250
): 
    """
    Itera por AÑO y por cada TÉRMINO de género.
    - Limita el total global de canciones a max_total_songs.
    - Limita el máximo por año a max_songs_per_year.
    """
    spotify_data = []
    total_songs = 0  # contador global

    for year in years:
        year_range = f'{year}-{year}'
        songs_in_year = 0  # contador por año
        print(f"\n--- INICIANDO EXTRACCIÓN PARA EL AÑO {year} ---")

        for term in query_terms:
            # Si ya se alcanzó el límite global o el del año, paramos
            if total_songs >= max_total_songs:
                print("\n⚠️ Límite global de canciones alcanzado. Deteniendo extracción.")
                return pd.DataFrame(spotify_data)

            if songs_in_year >= max_songs_per_year:
                print(f"⚠️ Límite de canciones para el año {year} alcanzado ({songs_in_year}). Pasando al siguiente año.")
                break

            query = f'genre:"{term}" year:{year_range}'
            print(f"   -> Buscando con término: {term}")

            try:
                results = sp.search(q=query, type='track', limit=50)
                total_resultados_api = results['tracks']['total'] if results and 'tracks' in results else 0
                print(f"   -> Spotify reporta {total_resultados_api} resultados para '{term}'.")

                while results and total_songs < max_total_songs and songs_in_year < max_songs_per_year:
                    for track in results['tracks']['items']:
                        track_id = track['id']

                        # Evitar duplicados
                        if any(d.get('ID_Spotify') == track_id for d in spotify_data):
                            continue

                        if total_songs >= max_total_songs or songs_in_year >= max_songs_per_year:
                            break

                        data_row = {
                            'Artista': track['artists'][0]['name'],
                            'Género musical': f'reggaeton (Origen: {term})',
                            'Tipo': 'Canción',
                            'Nombre': track['name'],
                            'Año de lanzamiento': year,
                            'ID_Spotify': track_id,
                            'ID_Album': track['album']['id']
                        }
                        spotify_data.append(data_row)
                        total_songs += 1
                        songs_in_year += 1

                    # Paginación
                    if results['tracks']['next'] and total_songs < max_total_songs and songs_in_year < max_songs_per_year:
                        results = sp.next(results['tracks'])
                    else:
                        results = None

                    time.sleep(0.5)

                print(f"      -> Acumulado año {year}: {songs_in_year} | Total global: {total_songs}")

            except Exception as e:
                print(f"   -> Error al buscar con término '{term}': {e}")
                time.sleep(2)

    return pd.DataFrame(spotify_data)


# --- LLAMADA ---
if sp:
    df_spotify_raw = extract_spotify_reggaeton_ampliada(
        sp,
        YEARS,
        query_terms,
        max_total_songs=500,    # máximo total
        max_songs_per_year=250  # máximo por año
    )
    print("\n--- RESULTADO FINAL DE REGGAETON (ESTRATEGIA ROBUSTA) ---")
    print(f"Total de registros extraídos: {len(df_spotify_raw)}")
    print(df_spotify_raw.head(10))


NameError: name 'sp' is not defined

In [9]:
# --- LLAMADA AL CÓDIGO ---

if sp:
    # Pasamos la lista de términos como argumento
    df_spotify_raw = extract_spotify_data_reggaeton_ampliada(sp, YEARS, query_terms, max_total_songs=500)
    print("\n--- RESULTADO FINAL DE REGGAETON (ESTRATEGIA ROBUSTA) ---")
    print(f"Total de registros extraídos: {len(df_spotify_raw)} (Nota: este número ya elimina duplicados entre términos)")
    print(df_spotify_raw.head(10))


--- INICIANDO EXTRACCIÓN PARA EL AÑO 2000 ---
   -> Buscando con término: reggaeton
   -> Spotify reporta 60 resultados para 'reggaeton'.
      -> Añadidas 60 canciones de 'reggaeton'.
   -> Buscando con término: regueton
   -> Spotify reporta 1 resultados para 'regueton'.
      -> Añadidas 1 canciones de 'regueton'.
   -> Buscando con término: urbano latino
   -> Spotify reporta 3 resultados para 'urbano latino'.
      -> Añadidas 2 canciones de 'urbano latino'.
   -> Buscando con término: perreo
   -> Spotify reporta 16 resultados para 'perreo'.
      -> Añadidas 5 canciones de 'perreo'.
   -> Buscando con término: reggaeton pop
   -> Spotify reporta 1 resultados para 'reggaeton pop'.
      -> Añadidas 0 canciones de 'reggaeton pop'.
   -> Buscando con término: latin urban
   -> Spotify reporta 50 resultados para 'latin urban'.
      -> Añadidas 46 canciones de 'latin urban'.
   -> Buscando con término: pop latino
   -> Spotify reporta 100 resultados para 'pop latino'.
      -> Añad

In [10]:
# VERIFICACIÓN DE CANCIONES
print(f"Número de canciones encontradas en df_spotify_raw: {len(df_spotify_raw)}")
print(f"Primeras filas (head) de df_spotify_raw:\n{df_spotify_raw.head()}")

Número de canciones encontradas en df_spotify_raw: 1036
Primeras filas (head) de df_spotify_raw:
  Artista                 Género musical     Tipo       Nombre  \
0  Dj Joe  Reggaeton (Origen: reggaeton)  Canción      Orgasmo   
1  Dj Joe  Reggaeton (Origen: reggaeton)  Canción    Mai Grita   
2  Dj Joe  Reggaeton (Origen: reggaeton)  Canción  Que Melones   
3  Dj Joe  Reggaeton (Origen: reggaeton)  Canción      Mujeres   
4  Jairos  Reggaeton (Origen: reggaeton)  Canción        Siete   

   Año de lanzamiento              ID_Spotify                ID_Album  
0                2000  0E2PAktDgSw1qXEzOwHLMz  5KjlY2iLyL9qaexUbxTGeo  
1                2000  0YxFcX2Rhfp4mOaLFyTXbV  5KjlY2iLyL9qaexUbxTGeo  
2                2000  2XB1118qP1DLyE4deHZrqv  5KjlY2iLyL9qaexUbxTGeo  
3                2000  0rLJYPOKXUe2TfngVgH0eB  5KjlY2iLyL9qaexUbxTGeo  
4                2000  4wTee4RHe9rkK9fNCCHMi5  5VT7SiRbeHW2ELcs0wOoOj  


In [11]:
# --- CELDA COMPLETA: EXTRACCIÓN Y UNIFICACIÓN DE ÁLBUMES ---

# Asegúrate de que las librerías necesarias (pandas, time) y la variable 'sp' estén disponibles
import pandas as pd
import time
# Nota: La variable 'sp' debe estar definida por la Celda 1.

# -------------------------------------------------------------------
# BLOQUE 1: DEFINICIÓN DE LA FUNCIÓN
# -------------------------------------------------------------------

def extract_album_details(sp, album_ids):
    """
    Extrae los metadatos completos (nombre, año, artista) de los álbumes.
    """
    album_details = []
    
    # El set ya se crea cuando se llama a la función con album_ids_list.
    unique_album_ids = album_ids
    
    print(f"\n--- INICIANDO EXTRACCIÓN DE METADATOS DE {len(unique_album_ids)} ÁLBUMES ---")
    
    for album_id in unique_album_ids:
        try:
            album_info = sp.album(album_id) 
            
            data_row = {
                'ID_Album': album_id,
                'Nombre_Album': album_info['name'],
                'Año_Lanzamiento_Album': album_info['release_date'][:4], # Tomamos solo el año
                'Artista_Principal_Album': album_info['artists'][0]['name'],
                'Tipo_Lanzamiento': album_info['album_type'] 
            }
            album_details.append(data_row)
            time.sleep(0.2) # Pausa para no saturar la API
            
        except Exception as e:
            # Capturamos errores como álbumes que han sido eliminados de Spotify
            print(f"   -> Error al extraer detalles del álbum {album_id}: {e}")
            pass
            
    print(f"✅ Extracción de {len(album_details)} álbumes completada.")
    return pd.DataFrame(album_details)

# -------------------------------------------------------------------
# BLOQUE 2: EJECUCIÓN Y FUSIÓN
# -------------------------------------------------------------------

# Verificación de que el DataFrame de canciones está disponible
if 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    
    print("\n--- INICIANDO PROCESO DE UNIFICACIÓN DE ÁLBUMES ---")
    
    # 1. VERIFICACIÓN CRÍTICA: La columna 'ID_Album' debe existir.
    if 'ID_Album' not in df_spotify_raw.columns:
        print("❌ ERROR CRÍTICO: La columna 'ID_Album' NO se encontró. La Celda 3 falló al guardar el ID del álbum.")
    else:
        
        # 2. Obtener IDs únicos válidos de los álbumes
        album_ids_list = df_spotify_raw['ID_Album'].dropna().unique() 
        
        if len(album_ids_list) == 0:
            print("❌ LISTA VACÍA: Se encontraron canciones, pero 0 IDs de álbumes válidos. No se puede continuar.")
        else:
            print(f"✅ Se encontraron {len(album_ids_list)} IDs de álbumes únicos. Extrayendo ahora...")
            
            # 3. Ejecutar la función de extracción (Tardará unos minutos)
            df_album_details = extract_album_details(sp, album_ids_list)
            
            # 4. Fusionar (JOIN) los detalles del álbum con los datos de las canciones
            df_spotify_final = pd.merge(
                df_spotify_raw,
                df_album_details[['ID_Album', 'Nombre_Album', 'Año_Lanzamiento_Album', 'Artista_Principal_Album']],
                on='ID_Album',
                how='left' 
            )
            
            print(f"\n--- PROCESO DE ÁLBUMES COMPLETADO ---")
            print(f"DataFrame intermedio 'df_spotify_final' creado con {len(df_spotify_final)} filas.")
            print(df_spotify_final.head())
            
else:
    print("❌ ERROR: El DataFrame de canciones (df_spotify_raw) no está disponible. Vuelve a ejecutar la Celda 3.")


--- INICIANDO PROCESO DE UNIFICACIÓN DE ÁLBUMES ---
✅ Se encontraron 693 IDs de álbumes únicos. Extrayendo ahora...

--- INICIANDO EXTRACCIÓN DE METADATOS DE 693 ÁLBUMES ---
✅ Extracción de 693 álbumes completada.

--- PROCESO DE ÁLBUMES COMPLETADO ---
DataFrame intermedio 'df_spotify_final' creado con 1036 filas.
  Artista                 Género musical     Tipo       Nombre  \
0  Dj Joe  Reggaeton (Origen: reggaeton)  Canción      Orgasmo   
1  Dj Joe  Reggaeton (Origen: reggaeton)  Canción    Mai Grita   
2  Dj Joe  Reggaeton (Origen: reggaeton)  Canción  Que Melones   
3  Dj Joe  Reggaeton (Origen: reggaeton)  Canción      Mujeres   
4  Jairos  Reggaeton (Origen: reggaeton)  Canción        Siete   

   Año de lanzamiento              ID_Spotify                ID_Album  \
0                2000  0E2PAktDgSw1qXEzOwHLMz  5KjlY2iLyL9qaexUbxTGeo   
1                2000  0YxFcX2Rhfp4mOaLFyTXbV  5KjlY2iLyL9qaexUbxTGeo   
2                2000  2XB1118qP1DLyE4deHZrqv  5KjlY2iLyL9qaexUbxTG

In [ ]:
f2754b20f21948c69e02a7509d75f701


--- EXTRACCIÓN DETALLES LAST.FM ---

--- DATAFRAME FINAL DE REGGAETON LISTO PARA ALMACENAMIENTO ---
  Artista                 Género musical     Tipo       Nombre  \
0  Dj Joe  Reggaeton (Origen: reggaeton)  Canción      Orgasmo   
1  Dj Joe  Reggaeton (Origen: reggaeton)  Canción    Mai Grita   
2  Dj Joe  Reggaeton (Origen: reggaeton)  Canción  Que Melones   
3  Dj Joe  Reggaeton (Origen: reggaeton)  Canción      Mujeres   
4  Jairos  Reggaeton (Origen: reggaeton)  Canción        Siete   

   Año de lanzamiento              ID_Spotify                ID_Album  \
0                2000  0E2PAktDgSw1qXEzOwHLMz  5KjlY2iLyL9qaexUbxTGeo   
1                2000  0YxFcX2Rhfp4mOaLFyTXbV  5KjlY2iLyL9qaexUbxTGeo   
2                2000  2XB1118qP1DLyE4deHZrqv  5KjlY2iLyL9qaexUbxTGeo   
3                2000  0rLJYPOKXUe2TfngVgH0eB  5KjlY2iLyL9qaexUbxTGeo   
4                2000  4wTee4RHe9rkK9fNCCHMi5  5VT7SiRbeHW2ELcs0wOoOj   

  Playcount_LastFM Listeners_LastFM  
0           187135       

In [13]:
# --- BLOQUE DE FUSIÓN Y GUARDADO CORREGIDO ---

# 1. Aseguramos que tenemos df_spotify_final y df_lastfm en memoria.

if 'df_spotify_final' in locals() and 'df_lastfm' in locals():
    
    # Lista de columnas a fusionar (¡EXCLUYE Artistas_Similares!)
    lastfm_cols_to_merge = ['Artista', 'Biografia_Resumen', 'Playcount_LastFM', 'Listeners_LastFM']
    
    # 2. Re-ejecutar el merge (fusión)
    df_proyecto_final = pd.merge(
        df_spotify_final, 
        # Seleccionamos solo las columnas que sabemos que existen:
        df_lastfm[lastfm_cols_to_merge], # <-- ¡CRUCIAL!
        on='Artista', 
        how='left'
    )
    
    # 3. Guardar el archivo CSV (Tu Celda 5 original)
    output_filename = 'reggaeton_data_2000_2024.csv'
    
    df_proyecto_final.to_csv(
        output_filename, 
        index=False,        
        encoding='utf-8'    
    )
    
    print(f"\n🎉 ¡FASE 1 FINALIZADA CON ÉXITO! 🎉")
# ...


🎉 ¡FASE 1 FINALIZADA CON ÉXITO! 🎉
